In [1]:
import pandas as pd 
import numpy as np
import geopandas as gpd
import plotly.offline as pyo 
import plotly.graph_objs as go
import plotly.figure_factory as ff
from plotly import tools
from plotly.offline import init_notebook_mode
pd.set_option('display.max_rows', 100)

In [2]:
conceito = pd.read_excel('../dados/Conceito_Enade_Todas_as_IES_2019_XLXS.xlsx')

In [3]:
conceito['UFBA'] = np.where(conceito['Sigla da IES'] == "UFBA", True, False)
['Pública Federal', 'Pública Estadual','Privada sem fins lucrativos', 'Pública Municipal', 'Privada com fins lucrativos']
conceito['PUB'] = np.where( np.logical_or( conceito['Categoria Administrativa'] == 'Privada sem fins lucrativos', conceito['Categoria Administrativa'] == 'Privada com fins lucrativos'), False, True)

In [4]:
fltr= conceito['Área de Avaliação']== 'ARQUITETURA E URBANISMO'
arq = conceito.loc[fltr]

In [5]:
conceito.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8368 entries, 0 to 8367
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Ano                               8368 non-null   int64  
 1   Código da Área                    8368 non-null   int64  
 2   Área de Avaliação                 8368 non-null   object 
 3   Código da IES                     8368 non-null   int64  
 4   Nome da IES                       8368 non-null   object 
 5   Sigla da IES                      7493 non-null   object 
 6   Organização Acadêmica             8368 non-null   object 
 7   Categoria Administrativa          8368 non-null   object 
 8   Código do Curso                   8368 non-null   int64  
 9   Modalidade de Ensino              8368 non-null   object 
 10  Código do Município               8368 non-null   int64  
 11  Município do Curso                8368 non-null   object 
 12  Sigla 

In [6]:
fltr = conceito['Sigla da IES'] == 'UFBA'
conc_UFBA = conceito.loc[fltr]

In [7]:
arq_ord = arq.sort_values(by=['Conceito Enade (Contínuo)']).reset_index()
print(arq_ord.shape)
ufba_pos = arq_ord['Sigla da IES'] == 'UFBA'
arq_ord.loc[ufba_pos]

(400, 25)


,index,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,...,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação,UFBA,PUB
356,1599,2019,21,ARQUITETURA E URBANISMO,578,UNIVERSIDADE FEDERAL DA BAHIA,UFBA,Universidade,Pública Federal,117044,...,54,52.3685,3.9361,65.4167,3.5228,3.6262,4,NaN,True,True
365,1598,2019,21,ARQUITETURA E URBANISMO,578,UNIVERSIDADE FEDERAL DA BAHIA,UFBA,Universidade,Pública Federal,13266,...,147,56.3361,4.5631,65.6735,3.5592,3.8102,4,NaN,True,True


In [8]:
arq_faixa = arq.groupby(['Conceito Enade (Faixa)']).count().reset_index()

arq_faixa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Conceito Enade (Faixa)            6 non-null      object
 1   Ano                               6 non-null      int64 
 2   Código da Área                    6 non-null      int64 
 3   Área de Avaliação                 6 non-null      int64 
 4   Código da IES                     6 non-null      int64 
 5   Nome da IES                       6 non-null      int64 
 6   Sigla da IES                      6 non-null      int64 
 7   Organização Acadêmica             6 non-null      int64 
 8   Categoria Administrativa          6 non-null      int64 
 9   Código do Curso                   6 non-null      int64 
 10  Modalidade de Ensino              6 non-null      int64 
 11  Código do Município               6 non-null      int64 
 12  Município do Curso        

In [9]:
data = [go.Bar(
    x=arq_faixa['Conceito Enade (Faixa)'],
    y=arq_faixa['Ano'],
    marker=dict(color=arq_faixa['Ano'],  colorscale ='tealgrn')
)]
layout = go.Layout(
    width=750, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Brasil por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarBrCondceitoFaixa.html')

'BarBrCondceitoFaixa.html'

In [10]:
arq_faixa.sort_values(by='Conceito Enade (Faixa)', inplace =True)

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa['Ano'], labels = arq_faixa['Conceito Enade (Faixa)'], marker=dict( colors=arq_faixa['Ano'])
)

layout={ 'width' : 600, 'height' :500,
        'showlegend': False,
        'title': "Cursos de Arquitetura e Urbanismo do Brasil por conceito (Faixa)"
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieBrCondceitoFaixa.html')

'PieBrCondceitoFaixa.html'

In [11]:
uf = pd.read_csv('../dados/brasil.csv', header = 0)
uf.head()

,SIGLA,ESTADO,CAPITAL,REGIAO
0,AC,ACRE,RIO BRANCO,NORTE
1,AM,AMAZONAS,MANAUS,NORTE
2,AP,AMAPÁ,MACAPÁ,NORTE
3,PA,PARÁ,BELÉM,NORTE
4,RO,RONDÔNIA,PORTO VELHO,NORTE


In [12]:
fltr = uf['REGIAO'] == 'NORDESTE'
ne = uf[fltr]

In [13]:
listNe = ne['SIGLA'].unique()

In [14]:
fltr = uf['REGIAO'].isin(['NORDESTE', 'NORTE'])
nne = uf[fltr]

listNNe = nne['SIGLA'].unique()
listNNe

array(['AC', 'AM', 'PA', 'RO', 'RR', 'TO', 'AL', 'BA', 'CE', 'MA', 'PB',
       'PE', 'PI', 'RN', 'SE'], dtype=object)

In [15]:
fltr = arq['Sigla da UF'].isin(listNe)
arq_ne = arq.loc[fltr]
arq_ne.head()

,Ano,Código da Área,Área de Avaliação,Código da IES,Nome da IES,Sigla da IES,Organização Acadêmica,Categoria Administrativa,Código do Curso,Modalidade de Ensino,...,Nº de Concluintes Participantes,Nota Bruta - FG,Nota Padronizada - FG,Nota Bruta - CE,Nota Padronizada - CE,Conceito Enade (Contínuo),Conceito Enade (Faixa),Observação,UFBA,PUB
1418,2019,21,ARQUITETURA E URBANISMO,3,UNIVERSIDADE FEDERAL DE SERGIPE,UFS,Universidade,Pública Federal,99444,Educação Presencial,...,54,53.2852,4.0810,64.6815,3.4187,3.5843,4,NaN,False,True
1420,2019,21,ARQUITETURA E URBANISMO,5,UNIVERSIDADE FEDERAL DO PIAUÍ,UFPI,Universidade,Pública Federal,517,Educação Presencial,...,76,51.8487,3.8540,69.6750,4.1257,4.0578,5,NaN,False,True
1425,2019,21,ARQUITETURA E URBANISMO,11,UNIVERSIDADE CATÓLICA DE PERNAMBUCO,UNICAP,Universidade,Privada sem fins lucrativos,52971,Educação Presencial,...,97,43.4278,2.5232,59.5474,2.6919,2.6497,3,NaN,False,False
1519,2019,21,ARQUITETURA E URBANISMO,352,CENTRO UNIVERSITÁRIO DE JOÃO PESSOA,UNIPÊ,Centro Universitário,Privada com fins lucrativos,38551,Educação Presencial,...,204,43.2373,2.4931,57.8740,2.4549,2.4645,3,NaN,False,False
1526,2019,21,ARQUITETURA E URBANISMO,385,UNIVERSIDADE SALVADOR,UNIFACS,Universidade,Privada com fins lucrativos,20575,Educação Presencial,...,440,44.4668,2.6874,56.4891,2.2589,2.3660,3,NaN,False,False


In [16]:
arq_faixa_ne = arq_ne.groupby(['Conceito Enade (Faixa)']).count().reset_index()

arq_faixa_ne.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 24 columns):
 #   Column                            Non-Null Count  Dtype 
---  ------                            --------------  ----- 
 0   Conceito Enade (Faixa)            5 non-null      object
 1   Ano                               5 non-null      int64 
 2   Código da Área                    5 non-null      int64 
 3   Área de Avaliação                 5 non-null      int64 
 4   Código da IES                     5 non-null      int64 
 5   Nome da IES                       5 non-null      int64 
 6   Sigla da IES                      5 non-null      int64 
 7   Organização Acadêmica             5 non-null      int64 
 8   Categoria Administrativa          5 non-null      int64 
 9   Código do Curso                   5 non-null      int64 
 10  Modalidade de Ensino              5 non-null      int64 
 11  Código do Município               5 non-null      int64 
 12  Município do Curso        

In [17]:
data = [go.Bar(
    x=arq_faixa_ne['Conceito Enade (Faixa)'],
    y=arq_faixa_ne['Ano'],
    marker=dict(color=arq_faixa_ne['Ano'],  colorscale ='peach')
)]
layout = go.Layout(
    width=750, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Nordeste por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarNeCondceitoFaixa.html')

'BarNeCondceitoFaixa.html'

In [18]:
arq_faixa_ne  = arq_faixa_ne.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_ne['Ano'], labels = arq_faixa_ne['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo do Nodeste por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieNeCondceitoFaixa.html')

'PieNeCondceitoFaixa.html'

In [19]:
fltr = arq['Sigla da UF'].isin(listNNe)
arq_nne = arq.loc[fltr]
arq_faixa_nne = arq_nne.groupby(['Conceito Enade (Faixa)']).count().reset_index()

In [20]:
data = [go.Bar(
    x=arq_faixa_nne['Conceito Enade (Faixa)'],
    y=arq_faixa_nne['Ano'],
    marker=dict(color=arq_faixa_nne['Ano'],  colorscale ='portland')
)]
layout = go.Layout(
    width=800, height=400,
    title='Número de cursos de Arquitetura e Urbanismo do Norte-Nordeste por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarNNeCondceitoFaixa.html')

'BarNNeCondceitoFaixa.html'

In [21]:
arq_faixa_nne  = arq_faixa_nne.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_nne['Ano'], labels = arq_faixa_nne['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo do Nodeste por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieNNeCondceitoFaixa.html')

'PieNNeCondceitoFaixa.html'

In [22]:
fltr = arq['Sigla da UF'] == "BA"
arq_BA = arq.loc[fltr]
arq_faixa_BA = arq_BA.groupby(['Conceito Enade (Faixa)']).count().reset_index()

In [23]:
data = [go.Bar(
    x=arq_faixa_BA['Conceito Enade (Faixa)'],
    y=arq_faixa_BA['Ano'],
    marker=dict(color=arq_faixa_BA['Ano'],  colorscale ='cividis')
)]
layout = go.Layout(
    width=800, height=400,
    title='Número de cursos de Arquitetura e Urbanismo da Bahia por conceito (Faixa)',
    yaxis=dict(
        title_text="Número de cursos"
    ),
     xaxis=dict(
        title_text="Conceito (Faixa)"
    ),
     hovermode = 'x'
)
fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='BarBACondceitoFaixa.html')

'BarBACondceitoFaixa.html'

In [24]:
arq_faixa_BA  = arq_faixa_BA.sort_values(by='Conceito Enade (Faixa)')

colors = ['chocolate', 'coral', 'cornflowerblue',
                'goldenrod', 'crimson', 'chartreuse']

data = go.Pie( values=arq_faixa_BA['Ano'], labels = arq_faixa_BA['Conceito Enade (Faixa)'], marker=dict( colors=colors)
)

layout={ 'width' : 650, 'height' :600,
        'showlegend': True,
        'title': "Cursos de Arquitetura e Urbanismo da Bahia por conceito (Faixa)"
        
        }

fig = go.Figure(data=data, layout=layout)
pyo.plot(fig, filename='PieBACondceitoFaixa.html')

'PieBACondceitoFaixa.html'

In [25]:
arq['Conceito Enade (Contínuo)'] = arq['Conceito Enade (Contínuo)'].fillna(0.0)
data = [go.Scatter( x = arq['Nome da IES'], y = arq['Conceito Enade (Contínuo)'], mode='markers', 
customdata = arq[['Categoria Administrativa','Sigla da UF','Código do Curso']],
name='2019',

hovertemplate =
    '<b>IES</b>: %{x}'+
    '<br><b>Coneito</b>(contínuo): %{y:.4f}'+
    '<br><b>Categoria Administrativa</b>: %{customdata[0]}'+
    '<br><b>UF</b>: %{customdata[1]}' +
    '<br><b>Código do Curso</b>: %{customdata[2]}'
    

    ,

marker=dict(
        symbol = np.where( arq['PUB'] , 0 , 2),
        color=arq['Conceito Enade (Contínuo)'] , colorscale ='portland',
        size= arq['Conceito Enade (Contínuo)'] *.5 + 10,
        line = dict( 
                width = np.where( arq['UFBA'] , 3, 1), color = np.where( arq['UFBA'] , 'black', 'darkslategray'))
        )


)]

layout={ 'width' : 1600, 'height' :750,
        'showlegend': False,
        'title': "Diagrama de dispersão: Cursos de Arquitetura e Urbanismo Do Brasil por conceito (continuo)",
        #'hovermode' : "x unified",
        'xaxis': { 'visible': False}
        }

fig = go.Figure(data=data, layout = layout)
pyo.plot(fig, filename='Continuo_disp_br.html')

'Continuo_disp_br.html'